In [1]:
from utils import *
import numpy as np
import matplotlib.pyplot as plt
import pyqpanda as pq
import pyvqnet as pv
import random

%matplotlib inline

In [2]:
train_data = Dataset(
    "data/cifar-10-batches-py/data_batch_1",
    "data/cifar-10-batches-py/data_batch_2",
    "data/cifar-10-batches-py/data_batch_3",
    "data/cifar-10-batches-py/data_batch_4",
    "data/cifar-10-batches-py/data_batch_5",
)
train_data.enhance()
test_data = Dataset("data/cifar-10-batches-py/test_batch")

In [3]:
class Model(pv.nn.module.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = pv.nn.Conv2D(3, 32, (3, 3))
        self.norm1 = pv.nn.BatchNorm2d(32)
        self.conv2 = pv.nn.Conv2D(32, 32, (3, 3))
        self.norm2 = pv.nn.BatchNorm2d(32)
        self.pool2 = pv.nn.MaxPool2D((2, 2), (2, 2))
        self.li = pv.nn.Linear(32 * 14 * 14, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.pool2(x)
        # print(x.shape)
        x = pv.tensor.flatten(x, 1)
        x = self.li(x)
        return x

In [4]:
epoch = 100
batch = 16
holdout = 10000

model = Model()
print(sum(p.numel() for p in model.parameters()))

73002


In [5]:
X, Y = train_data.getdatas()
los = pv.nn.loss.CrossEntropyLoss()
opt = pv.optim.Adam(model.parameters())

acc_train = []
acc_test = []

for e in range(1, epoch + 1):
    model.train()

    correct_tot = 0
    for I, (x, y) in enumerate(
        pv.data.data_generator(
            X[:-holdout], Y[:-holdout], batch_size=batch, shuffle=True
        )
    ):
        i = I + 1
        # print(i)
        opt.zero_grad()
        y_pred = model(x)
        loss = los(y, y_pred)
        correct = sum(y_pred.argmax(1, False).to_numpy() == y)
        acc = correct / y.shape[0]
        correct_tot += correct
        loss.backward()
        opt._step()
        if i % 100 == 0:
            print(
                f"epoch {e}/{epoch} \t batch {batch*i}/{Y.shape[0]-holdout} \t loss {loss.item():.2f} \t accuracy {acc:.2f}"
            )

    model.eval()

    acc = correct_tot / (Y.shape[0] - holdout)
    print(f"********** train: epoch {e}/{epoch} \t accuracy {100*acc:.2f}% **********")
    acc_train.append(acc)

    correct_tot = 0
    for x, y in pv.data.data_generator(
        X[-holdout:], Y[-holdout:], batch_size=batch, shuffle=False
    ):
        y_pred = model(x)
        correct_tot += sum(y_pred.argmax(1, False).to_numpy() == y)

    acc = correct_tot / holdout
    print(f"********** test: epoch {e}/{epoch} \t accuracy {100*acc:.2f}% **********")
    acc_test.append(acc)

    # opt = pv.optim.SGD(model.parameters(), 0.01 * (1 - acc))

epoch 1/100 	 batch 1600/90000 	 loss 5.25 	 accuracy 0.31
epoch 1/100 	 batch 3200/90000 	 loss 1.98 	 accuracy 0.31
epoch 1/100 	 batch 4800/90000 	 loss 1.92 	 accuracy 0.44
epoch 1/100 	 batch 6400/90000 	 loss 1.63 	 accuracy 0.56
epoch 1/100 	 batch 8000/90000 	 loss 1.65 	 accuracy 0.44
epoch 1/100 	 batch 9600/90000 	 loss 1.60 	 accuracy 0.38
epoch 1/100 	 batch 11200/90000 	 loss 1.72 	 accuracy 0.38
epoch 1/100 	 batch 12800/90000 	 loss 1.54 	 accuracy 0.31
epoch 1/100 	 batch 14400/90000 	 loss 2.20 	 accuracy 0.25
epoch 1/100 	 batch 16000/90000 	 loss 1.89 	 accuracy 0.25
epoch 1/100 	 batch 17600/90000 	 loss 1.79 	 accuracy 0.38
epoch 1/100 	 batch 19200/90000 	 loss 1.68 	 accuracy 0.56
epoch 1/100 	 batch 20800/90000 	 loss 1.73 	 accuracy 0.19
epoch 1/100 	 batch 22400/90000 	 loss 2.42 	 accuracy 0.12
epoch 1/100 	 batch 24000/90000 	 loss 1.84 	 accuracy 0.38
epoch 1/100 	 batch 25600/90000 	 loss 1.55 	 accuracy 0.50
epoch 1/100 	 batch 27200/90000 	 loss 1.49 	 

KeyboardInterrupt: 

In [6]:
pv.utils.storage.save_parameters(model.state_dict(), "train9.model")

In [7]:
m = Model()
m.load_state_dict(pv.utils.storage.load_parameters("train9.model"))

In [8]:
m.eval()
X, Y = test_data.getdatas()
correct_tot = 0
for x, y in pv.data.data_generator(X, Y, batch_size=batch, shuffle=False):
    y_pred = model(x)
    correct_tot += sum(y_pred.argmax(1, False).to_numpy() == y)
acc = correct_tot / Y.shape[0]
print(f"********** eval: accuracy {100*acc:.2f}% **********")

********** eval: accuracy 51.54% **********
